In [ ]:
import os
import csv
from googleapiclient.discovery import build
from datetime import datetime, timedelta
import pandas as pd
import isodate


#set your API key
api_key = "---"
youtube = build("youtube", "v3", developerKey=api_key)
#here is request parameters
publishedAfter = datetime.strptime("2006-01-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ")
publishedBefore = datetime.strptime("2023-05-16T23:59:59Z", "%Y-%m-%dT%H:%M:%SZ")
max_results = 50
total_results = 0
page_token = ""
file_counter = 1
video_counter = 0
#Sets the next "publishedAfter" date for the search, considering the maximum results per day
while publishedAfter < publishedBefore:
    next_publishedAfter = publishedAfter + timedelta(days=1)
    if next_publishedAfter > publishedBefore:
        next_publishedAfter = publishedBefore
#Executes a search request to the YouTube API to retrieve videos related to "climate change" within the specified date range
    search_response = youtube.search().list(
        q="climate change",
        type="video",
        part="id",
        order='title',
        relevanceLanguage='en',
        publishedAfter=publishedAfter.strftime("%Y-%m-%dT%H:%M:%SZ"),
        publishedBefore=next_publishedAfter.strftime("%Y-%m-%dT%H:%M:%SZ"),
        maxResults=max_results,
        pageToken=page_token
    ).execute()
#Checks if there are any search results. If not, breaks the loop
    if 'items' not in search_response:
        break
#Retrieves the search results and updates the total number of results
    search_results = search_response.get("items", [])
    total_results += len(search_results)

#Initializes the CSV file for writing the search results if it's the first video in the file
    tags = []
    if video_counter == 0:
        filename = f"May/search_results_{file_counter}.csv"
        with open(filename, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)          
# write header of csv file
            writer.writerow(
                ['title', 'channelId', 'channelTitle', 'videoId', 'tags', 'description', 'publishDate', 'url',
                 'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'dislikeCount', 'commentCount'])
            for result in search_results:
                video_id = result["id"]["videoId"]

# After get Video ID, the code makes an API request to retrieve the video details using the youtube.videos().list() method
                video_response = youtube.videos().list(
                    part="contentDetails,statistics, snippet",
                    id=video_id
                ).execute()
#It checks if there are video items in the response and continues to the next iteration if not
                if 'items' not in video_response:
                    continue
                video_results = video_response.get("items", [])

                if not video_results:
                    continue

                video = video_results[0]

                title = video["snippet"]["title"]
                channel_id = video['snippet']['channelId']
                channel_title = video['snippet']['channelTitle']
                tags = video['snippet'].get("tags", None)
                description = video["snippet"]["description"]
                publish_date = video["snippet"]["publishedAt"]
                url = "https://www.youtube.com/watch?v=" + video_id
                thumbnail_high = video['snippet']['thumbnails']['high'].get('url', 0)
                duration_str = video['contentDetails'].get('duration', '')
                duration = isodate.parse_duration(duration_str)

                # Convert to a timedelta object
                duration_timedelta = timedelta(seconds=duration.total_seconds())
                view_count = video['statistics'].get('viewCount', 0)
                like_count = video['statistics'].get('likeCount', 0)
                dislike_count = video['statistics'].get('dislikeCount', 0)
                commentCount = video['statistics'].get('commentCount', 0)
#according to the header, write rows
                writer.writerow([title, channel_id, channel_title, video_id, tags, description,
                                 publish_date, url, thumbnail_high, duration_timedelta, view_count,
                                 like_count, dislike_count, commentCount])
#The code then increments the counters video_counter and file_counter based on the number of videos processed
#If video_counter reaches 500, it increments file_counter and resets video_counter to 0(that means if the rows reach to 500, it will create new csv file and start write)

                video_counter += 1
                if video_counter == 500:
            
#This part of the code handles writing the video details to a CSV file when the video_counter reaches 500
                    file_counter += 1
                    video_counter = 0
    else:
        filename = f"May/search_results_{file_counter}.csv"
        with open(filename, mode="a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            for result in search_results:
                video_id = result["id"]["videoId"]

                # Get video details
                video_response = youtube.videos().list(
                    part="contentDetails,statistics, snippet",
                    id=video_id
                ).execute()

                if 'items' not in video_response:
                    continue

                video_results = video_response.get("items", [])

                if not video_results:
                    continue

                video = video_results[0]

                title = video["snippet"]["title"]
                channel_id = video['snippet']['channelId']
                channel_title = video['snippet']['channelTitle']
                tags = video['snippet'].get('tags', None)
                description = video["snippet"]["description"]
                publish_date = video["snippet"]["publishedAt"]
                url = "https://www.youtube.com/watch?v=" + video_id
                thumbnail_high = video['snippet']['thumbnails']['high'].get('url', 0)
                duration_str = video['contentDetails'].get('duration', '')
                duration = isodate.parse_duration(duration_str)

                # Convert to a timedelta object
                duration_timedelta = timedelta(seconds=duration.total_seconds())
                view_count = video['statistics'].get('viewCount', 0)
                like_count = video['statistics'].get('likeCount', 0)
                dislike_count = video['statistics'].get('dislikeCount', 0)
                commentCount = video['statistics'].get('commentCount', 0)
                writer.writerow([title, channel_id, channel_title, video_id, tags, description,
                                 publish_date, url, thumbnail_high, duration_timedelta, view_count,
                                 like_count, dislike_count, commentCount])

                video_counter += 1
                if video_counter == 500:
                    file_counter += 1
                    video_counter = 0
# Check if there are more pages of results
#If there are more pages, it updates the publishedAfter date and page_token for the next iteration of the loop
    if 'nextPageToken' not in search_response:
        publishedAfter = next_publishedAfter
        page_token = ""
    else:
        page_token = search_response['nextPageToken']
